# PostGIS

The city of San Francisco wants to assess the quality of restaurants in the city. Their data is scattered across multiple sources and incomplete.

They tasked you to help them assess it.

They would like to know what the most common violations are where they happen most frequently.

We will use a PostgreSQL database configured with the geo-spatial tool, PostGIS, to do this work.

# Three Database Concepts

1. CRUD
2. connection and cursor
3. transactions

## CRUD

| | SQL | RESTful API |
|:-:|:-:|:-:|
| create | `INSERT` | `POST` |
| read | `SELECT` | `GET` |
| update | `UPDATE` | `PUT` |
| delete | `DELETE` | `DELETE` |


### Correspondence to Python Classes

In [1]:
class Foo():
    def __init__(self, bar, baz):
        self.bar = int(bar)
        self.baz = int(baz)

This class would correspond to a SQL table.

           foo
    +-------+-------+               
    |  bar  |  baz  |
    +-------+-------+
    |  int  |  int  |
    +-------+-------+    

In [2]:
foo_1 = Foo(4,3)
foo_2 = Foo(-1,2)
foo_3 = Foo(0,-1)

## Connection and Cursor

We will use a connection object to define our connection to our database over a network.

We will use a cursor object to write to the database. The cursor is created by a class function of the connection object.

In [3]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

connection = pg2.connect(host='postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)

The cursor can be used to execute queries and then to fetch the results of the query.

In [4]:
cursor.execute("SELECT * FROM business LIMIT 2;")

In [5]:
result = cursor.fetchall()

In [6]:
connection.close()

Because we used the `cursor_factory=RealDictCursor` argument, the results return a list of dictionary objects from the database. 

The advantage of this is that we can display these results easily with a DataFrame.


In [7]:
result

[{'address': '033 BELDEN PL',
  'city': 'San Francisco',
  'id': 10,
  'latitude': 37.791116,
  'longitude': -122.403816,
  'name': 'TIRAMISU KITCHEN',
  'phone_number': '+14154217044',
  'postal_code': 94104,
  'state': 'CA'},
 {'address': '1200 VAN NESS AVE, 3RD FLOOR',
  'city': 'San Francisco',
  'id': 19,
  'latitude': 37.786848,
  'longitude': -122.421547,
  'name': 'NRGIZE LIFESTYLE CAFE',
  'phone_number': '+14157763262',
  'postal_code': 94109,
  'state': 'CA'}]

In [8]:
import pandas as pd

In [9]:
connection = pg2.connect(host='postgis',
                         user='postgres',
                         database='postgres')
cursor = connection.cursor(cursor_factory=RealDictCursor)
cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


In [10]:
connection.close()

In [11]:
%%bash
pwd

/home/jovyan/ipynb


## `lib.db_helper`

Because we will keep using the connection-cursor pattern, we have written it into a sub-module.

In [20]:
from os import chdir
chdir('/home/jovyan')

In [21]:
import lib.db_helper as db

#### `db.connect_to_db()`

Contains our connection credentials and returns a connection and cursor. 

    def connect_to_db():
        con = pg2.connect(host='postgis',
                          dbname='postgres',
                          user='postgres')
        cur = con.cursor(cursor_factory=RealDictCursor)
        return con, cur

In [22]:
connection, cursor = db.connect_to_db()
result = cursor.execute("SELECT * FROM business LIMIT 2;")
pd.DataFrame(cursor.fetchall())

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


In [23]:
connection.close()

#### `db.query_to_dictionary()`

1. creates a connection and a cursor
1. uses the cursor to execute a query
1. if `fetch_res` is `True` it fetches the results, otherwise results are `None`
1. closes the connection and returns `results`


    def query_to_dictionary(query, fetch_res=True):
        con, cur = connect_to_db()
        cur.execute(query)
        if fetch_res:
            results = cur.fetchall()
        else:
            results = None
        con.close()
        return results

In [24]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""")

[{'address': '033 BELDEN PL',
  'city': 'San Francisco',
  'id': 10,
  'latitude': 37.791116,
  'longitude': -122.403816,
  'name': 'TIRAMISU KITCHEN',
  'phone_number': '+14154217044',
  'postal_code': 94104,
  'state': 'CA'},
 {'address': '1200 VAN NESS AVE, 3RD FLOOR',
  'city': 'San Francisco',
  'id': 19,
  'latitude': 37.786848,
  'longitude': -122.421547,
  'name': 'NRGIZE LIFESTYLE CAFE',
  'phone_number': '+14157763262',
  'postal_code': 94109,
  'state': 'CA'}]

In [25]:
db.query_to_dictionary("""SELECT * FROM business LIMIT 2;""", fetch_res=False)

#### `db.query_to_dataframe()`

Wraps `query_to_dictionary()` in a `pandas.DataFrame`.

    def query_to_dataframe(query):
        return DataFrame(query_to_dictionary(query))

In [26]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


## Transactions

As we begin working on "production" databases, it becomes critical to maintain the integrity of the data. 

Imagine a website with a database using the following table:

           page
    +-----------+---------------+               
    |  page_id  |  visit_count  |
    +-----------+---------------+               
    |    int    |      int      |
    +-----------+---------------+               

Suppose that a page with `page_id` `15` is visited nearly simultaneously. You could imagine the following sequence of events:

1. The first page visit occurs. 
2. The website requests to increment the `visit_count`.
3. The second page visit occurs.
4. Using a separate proces, the website requests to increment the `visit_count`.
5. The first increment is written to the database
6. The second increment is written to the database

How many visits have been recorded?

In order to ensure that no data is lost, PostgreSQL uses transactions.

- A transaction is begun with the keyword `BEGIN`.
- Changes to the database are made in memory.
- Changes to the database are validated. 
- If valid, the changes are made permanent with the keyword `COMMIT`
- If invalid, the changes are discarded with the keyword `ROLLBACK`


### Create new column in `business` table

To demonstrate how transactions work, we will write a SQL query to create a new column in our database.

In [27]:
db.query_to_dictionary("""
BEGIN;
ALTER TABLE business ADD COLUMN gpnt_location geometry(POINT,4326);
COMMIT;
""", fetch_res=False)

In [28]:
db.query_to_dataframe('SELECT * FROM business LIMIT 2;')

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,033 BELDEN PL,San Francisco,None,10,37.791116,-122.403816,TIRAMISU KITCHEN,+14154217044,94104,CA
1,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,None,19,37.786848,-122.421547,NRGIZE LIFESTYLE CAFE,+14157763262,94109,CA


We will want to populate this column, but the data is a bit messy as we will see in a moment. 

# Postgres Aggregate Functions

In this lesson, we will use [Postgres Aggregate](https://www.postgresql.org/docs/9.5/static/functions-aggregate.html) functions to explore data for businesses in San Francisco. 

Aggregate functions compute a single result from a set of input values.

Some aggregate functions include:

| Function | Description |
|:-:|:-:|
| `avg(expression)` | the average (arithmetic mean) of all input values |
| `count(*)` | number of input rows |
| `max(expression)` | maximum value of `expression` across all input values |
| `min(expression)` | minimum value of `expression` across all input values |
| `sum(expression)` | sum of `expression` across all input values |

# Clean the Data

We can use `COUNT` to count the number of rows in our `business` table. 

In [29]:
select_count_business = """
SELECT COUNT(*) FROM business
"""

In [30]:
print(select_count_business)


SELECT COUNT(*) FROM business



In [31]:
db.query_to_dataframe(select_count_business)

,count
0,6161


We can use `COUNT` with a `GROUP BY` clause to count the number of rows for each zip code. 

In [32]:
select_count_by_postal_code = """
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
"""

In [33]:
db.query_to_dataframe(select_count_by_postal_code)

,count,postal_code
0,1,0.0
1,1,84105.0
2,1,92672.0
3,2,94013.0
4,3,94014.0
5,1,94066.0
6,3,94101.0
7,447,94102.0
8,552,94103.0
9,132,94104.0


## Yikes! That is some messy data. 

### `WHERE` 

We next use the `WHERE` clause to restrict our count to just rows that have `latitude` and `longitude` data. 

In [38]:
select_count_business_non_null_lat_long = """
SELECT COUNT((latitude, longitude)) FROM business
WHERE latitude IS NOT NULL
AND longitude IS NOT NULL
"""

In [39]:
db.query_to_dataframe(select_count_business_non_null_lat_long)

,count
0,3749


Note that we might also have invalid data.

In [40]:
select_count_business_invalid_data = """
SELECT COUNT(*) FROM business
WHERE latitude = 0 AND longitude = 0;"""

In [41]:
db.query_to_dataframe(select_count_business_invalid_data)

,count
0,14


In [42]:
select_verify_no_edge_cases = """
SELECT COUNT(*) FROM business 
WHERE (latitude IS NULL AND longitude IS NOT NULL)
OR (latitude IS NOT NULL AND longitude IS NULL)
OR (latitude = 0 AND longitude != 0)
or (latitude != 0 AND longitude = 0)
"""

In [43]:
db.query_to_dataframe(select_verify_no_edge_cases)

,count
0,0


In [44]:
select_count_business_valid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_count_business_invalid = """
SELECT COUNT(*) FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
"""

In [45]:
db.query_to_dataframe(select_count_business_valid)

,count
0,3735


In [46]:
db.query_to_dataframe(select_count_business_invalid)

,count
0,2426


### Store Valid and Invalid Queries

We will also make use of subqueries. We will define two:

1. a subquery for all businesses with valid lat/long
1. a subquery for all businesses with invalid lat/long

#### Valid Business Query

In [48]:
select_business_valid = """
SELECT * FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)
"""

select_business_invalid = """
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
"""

In [49]:
print(db.query_to_dataframe(select_business_valid).shape)
db.query_to_dataframe(select_business_valid).sample(4)

(3735, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
3653,1501 CORTLAND AVE,San Francisco,None,70313,37.739764,-122.409475,SF LIVING GREENS,+14156029513,94110.0,CA
2693,1306 FULTON ST,San Francisco,None,59986,37.776838,-122.438563,TSUNAMI PANHANDLE,,94117.0,CA
3416,1559 FRANKLIN ST,San Francisco,None,68160,37.789073,-122.423784,GOURMET CAROUSEL,+14157712044,94109.0,CA
3548,958 GENEVA AVE,San Francisco,None,69026,37.715948,-122.440150,GM MARKET INC.,,94112.0,CA


In [50]:
print(db.query_to_dataframe(select_business_invalid).shape)
db.query_to_dataframe(select_business_invalid).sample(4)

(2426, 10)


,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
2369,333 O'FARRELL ST,San Francisco,None,86619,NaN,NaN,HERB N' KITCHEN,+14157711400,94102.0,CA
698,3149 MISSION ST,San Francisco,None,75415,NaN,NaN,TROP BON,+14156136559,94110.0,CA
2183,500 HAYES ST,San Francisco,None,85164,NaN,NaN,LA BOULANGERIE DE SAN FRANCISCO,+14154400356,94102.0,CA
1814,531 JACKSON ST,San Francisco,None,82578,NaN,NaN,TRESTLE,+14156903783,94133.0,CA


### Explore Postal Code and Lat/Long

First, let's get the postal codes with a count of less than 10. We will use a sub-query.

Let's use these subqueries to help us to clean the data. 

Note that we re-use `select_count_by_postal_code`.

In [51]:
print(select_count_by_postal_code)


SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code



In [63]:
"Hi I'm {} and {}.".format("Happy", "Blue")

"Hi I'm Happy and Blue."

In [70]:
select_postal_code_by_postal_code_less_than_10 = """
SELECT postal_code FROM 
    ({}) counts_by_postal_code
WHERE count < 10
""".format(select_count_by_postal_code)

print(select_postal_code_by_postal_code_less_than_10)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10



Note that we are using a subquery here. 

e.g. 

    SELECT * FROM (SUBQUERY) sub_query_name;
    

In [59]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,84105
2,92672
3,94013
4,94014
5,94066
6,94101
7,94120
8,94129
9,94130


We'll think of these as our "bad postal codes". We can think of the query we defined as a list of these codes. That query was

In [71]:
print(select_postal_code_by_postal_code_less_than_10)


SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10



Do we have (lat,long) pairs for these locations? Let's select all rows with one of these postal codes.

### `IN`

To do this we will use the SQL keyword `IN`. `IN` checks a column against a list of items.

e.g. 

    SELECT * FROM my_table WHERE my_column IN ('1,2,3,4,5');

# We will replace the list with our Sub-Query!! 

In [ ]:
broken_query = """
SELECT * FROM business WHERE 
"""

    SELECT * FROM business WHERE postal_code IN (
        SELECT postal_code FROM 
            (
            SELECT postal_code, COUNT(*) FROM business
            GROUP BY postal_code
            ORDER BY postal_code
        ) counts_by_postal_code
    WHERE count < 10
    ) 

In [72]:
select_business_where_bad_postal_code = """
SELECT * FROM business WHERE postal_code IN ({})""".format(select_postal_code_by_postal_code_less_than_10)
print(select_business_where_bad_postal_code)


SELECT * FROM business WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)


In [69]:
db.query_to_dataframe(select_business_where_bad_postal_code).sample(5)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
5,P.O. BOX 40381,San Francisco,None,37021,37.754018,-122.416439,GOZA-GOZA TACO,+14155746608,94140,CA
34,1900 MARKET ST,San Francisco,None,85459,NaN,NaN,ORBIT ROOM,+14153705584,94602,CA
8,"2 AVE OF PALMS BLDG 146, TREASURE ISLAND",San Francisco,None,38139,37.824494,-122.371798,ISLAND MARKET & DELI,+14158600303,94130,CA
22,101 BAYSHORE BLVD,San Francisco,None,74674,NaN,NaN,ELI'S HOT DOGS,+14158301168,94014,CA
19,"HUNTER'S POINT SHIPYARD, BLDG.110",San Francisco,None,71471,NaN,NaN,O'CACAO,+14159062574,94188,CA


Note that some of these have a lat and long. 

What we want is rows that have "bad postal codes" and no (lat, long) pair. 

### A Nested Query

To get these we will write a nested query that looks like this:

    SELECT * FROM (invalid_lat_long) WHERE postal_code IN (bad_postal_code_str)

In [73]:
bad_postal_codes = select_postal_code_by_postal_code_less_than_10

In [74]:
select_invalid_business_where_bad_postal_code = """
SELECT * 
FROM ({}) invalid_business
WHERE postal_code IN ({})
""".format(select_business_invalid, bad_postal_codes)
print(select_invalid_business_where_bad_postal_code)


SELECT * 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
) invalid_business
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)



In [75]:
db.query_to_dataframe(select_invalid_business_where_bad_postal_code)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,VARIOUS LOACATIONS (17),San Francisco,None,5755,None,None,J & J VENDING,+14156750910,94545,CA
1,VARIOUS LOCATIONS,San Francisco,None,5757,None,None,"RICO VENDING, INC",+14155836723,94066,CA
2,390 CLEMENTINA ST,San Francisco,None,6029,None,None,ALEXIS APARTMENTS SR. MEALS PROGRAM,+14154959541,941033148,CA
3,TREASURE ISLAND 888 AVENUE H,San Francisco,None,63091,None,None,MAYA WATERS,+14156137162,94130,CA
4,"150 04TH ST., TREASURE ISLAND SUITE 1",San Francisco,None,69804,None,None,ISLAND COVE MARKET,+14153912299,94130,CA
5,"HUNTER'S POINT SHIPYARD, BLDG.110",San Francisco,None,71471,None,None,O'CACAO,+14159062574,94188,CA
6,5383 CAPWELL,San Francisco,None,72127,None,None,REVOLUTION FOODS,,94621,CA
7,150 04TH ST SUITE 1,San Francisco,None,73799,None,None,ISLAND COVE MARKET,+14153912299,94130,CA
8,101 BAYSHORE BLVD,San Francisco,None,74674,None,None,ELI'S HOT DOGS,+14158301168,94014,CA
9,2769 LOMBARD ST,San Francisco,None,77759,None,None,PIZZERIA AVELLINO,+14157762500,94129,CA


#### Let's use `DELETE` to get rid of rows that have no lat, long and a bad zip

First, let's collect a list of business `id`s. 

In [76]:
select_invalid_business_id_where_bad_postal_code = """
SELECT id 
FROM ({}) q
WHERE postal_code IN ({})
""".format(select_business_invalid, select_postal_code_by_postal_code_less_than_10)
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)



In [77]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

,id
0,5755
1,5757
2,6029
3,63091
4,69804
5,71471
6,72127
7,73799
8,74674
9,77759


In [78]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)



We can use this subquery in our `DELETE` query.

Remember, that a **delete** action is a write action and needs to be handled as a transaction. We need to `BEGIN` and `COMMIT`. 

In [79]:
delete_invalid_business_bad_postal_code = """
BEGIN;
DELETE 
FROM business
WHERE id IN ({});
COMMIT;
""".format(select_invalid_business_id_where_bad_postal_code)
print(delete_invalid_business_bad_postal_code)


BEGIN;
DELETE 
FROM business
WHERE id IN (
SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)
);
COMMIT;



Remember, when we execute this we do not need to fetch results so we should set `fetch_res` to `False`.

In [80]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

IntegrityError: update or delete on table "business" violates foreign key constraint "fk_business_id" on table "inspection"
DETAIL:  Key (id)=(5755) is still referenced from table "inspection".


Let's have a look at the sql file used to define our database. 

```SQL
CREATE TABLE business (
    id INTEGER,
    name TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    phone_number TEXT,
    PRIMARY KEY (id));

CREATE TABLE inspection (
    business_id INTEGER,
    score INTEGER,
    date TIMESTAMP,
    type TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

CREATE TABLE legend (
    minimum_score INTEGER,
    maximum_score INTEGER,
    description TEXT);

CREATE TABLE violation (
    business_id INTEGER,
    date TIMESTAMP,
    description TEXT,
    CONSTRAINT fk_business_id
    FOREIGN KEY (business_id)
    REFERENCES business (id));

```

Note that we have foreign key `CONSTRAINT`s on the `inspection` and `violation` tables.  

This means that, in order to drop the businesses with "bad" data, we will need to drop any inspections and violations associated with these. 

### "Bad" Business Data

In [81]:
db.query_to_dataframe(select_invalid_business_id_where_bad_postal_code)

,id
0,5755
1,5757
2,6029
3,63091
4,69804
5,71471
6,72127
7,73799
8,74674
9,77759


In [82]:
print(select_invalid_business_id_where_bad_postal_code)


SELECT id 
FROM (
SELECT * FROM business
WHERE 
    (latitude IS NULL
     AND longitude IS NULL)
OR 
    (latitude = 0 
     AND longitude = 0)
) q
WHERE postal_code IN (
SELECT postal_code FROM 
    (
SELECT postal_code, COUNT(*) FROM business
GROUP BY postal_code
ORDER BY postal_code
) counts_by_postal_code
WHERE count < 10
)



In [83]:
select_violations_for_bad_biz = """
SELECT * FROM violation
WHERE business_id IN ({})
""".format(select_invalid_business_id_where_bad_postal_code)

In [84]:
db.query_to_dataframe(select_violations_for_bad_biz)

,business_id,date,description
0,6029,2014-03-13,Food safety certificate or food handler card n...
1,6029,2014-03-13,Unapproved or unmaintained equipment or utensi...
2,69804,2015-06-17,Low risk vermin infestation
3,69804,2015-06-17,Moderate risk food holding temperature
4,69804,2013-09-26,No thermometers or uncalibrated thermometers ...
5,69804,2013-09-26,Inadequate food safety knowledge or lack of ce...
6,71471,2013-05-02,Improper food storage
7,74674,2015-08-18,Inadequate and inaccessible handwashing facili...
8,74674,2015-08-18,Permit license or inspection report not posted...
9,74674,2015-08-18,Mobile food facility with unapproved operating...


### Delete Violations

In [85]:
select_count_violation = """
SELECT COUNT(*) FROM violation"""

In [86]:
db.query_to_dataframe(select_count_violation)

,count
0,39907


In [87]:
delete_violations_for_bad_biz = """
BEGIN;
DELETE FROM violation
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [88]:
db.query_to_dictionary(delete_violations_for_bad_biz, fetch_res=False)

In [89]:
db.query_to_dataframe(select_count_violation)

,count
0,39863


### Delete Inspections

In [90]:
select_count_inspections = """
SELECT COUNT(*) FROM inspection"""

In [91]:
db.query_to_dataframe(select_count_inspections)

,count
0,15429


In [92]:
delete_inspections_for_bad_biz = """
BEGIN;
DELETE FROM inspection
WHERE business_id IN ({});
COMMIT;""".format(select_invalid_business_id_where_bad_postal_code)

In [93]:
db.query_to_dictionary(delete_inspections_for_bad_biz, fetch_res=False)

In [94]:
db.query_to_dataframe(select_count_inspections)

,count
0,15408


### Delete Businesses

In [95]:
db.query_to_dataframe(select_count_business)

,count
0,6161


In [96]:
db.query_to_dictionary(delete_invalid_business_bad_postal_code, fetch_res=False)

In [97]:
db.query_to_dataframe(select_count_business)

,count
0,6138


### Pull Bad Postal Codes Again

In [98]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,92672
2,94013
3,94101
4,94120
5,94130
6,94140
7,94143
8,94513
9,94609


## Repair Bad Zip Codes
To repair the bad zip codes, we will attempt match the nearest points using GIS. 

To do this, we will first need to populate the `gpnt_location` column.

In [99]:
print(select_business_valid)


SELECT * FROM business
WHERE 
    (latitude IS NOT NULL
     AND longitude IS NOT NULL)
AND 
    (latitude != 0 
     AND longitude != 0)



In [102]:
db.query_to_dataframe(select_business_valid).sample(4)

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
160,2228 IRVING ST,San Francisco,0101000020E6100000E36C3A02B8E1424093533BC3D49E...,552,37.763428,-122.481736,LOI'S VIETNAMESE RESTAURANT,+14156615936,94122.0,CA
1767,2956 24TH ST,San Francisco,0101000020E610000013EF004F5AE04240FB75A73B4F9A...,7771,37.752756,-122.411086,NEW SWEETHEART BAKERY,+14158260876,94110.0,CA
609,4780 MISSION ST,San Francisco,0101000020E6100000D5EC815660DC4240F73B1405FA9B...,2309,37.721690,-122.437135,BURGER KING,+14154695518,94112.0,CA
3601,1624 POWELL ST,San Francisco,0101000020E610000096CD1C925AE64240E7374C34489A...,69383,37.799639,-122.410657,LA MEDITERRANEE,+14159212956,94133.0,CA


In [100]:
update_gpnt = """
BEGIN;
UPDATE business SET 
gpnt_location = ST_SetSRID(ST_MakePoint(latitude, longitude),4326);
COMMIT;
"""

In [101]:
db.query_to_dictionary(update_gpnt, fetch_res=False)

### Match points to the points with bad zip codes using a given radius

In [103]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,92672
2,94013
3,94101
4,94120
5,94130
6,94140
7,94143
8,94513
9,94609


In [104]:
def select_postal_code(postal_code):
    return """SELECT * FROM business WHERE postal_code = {}""".format(postal_code)

In [109]:
db.query_to_dataframe(select_postal_code(92672))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,92672,CA


In [110]:
def select_gpnt_for_postal_code(postal_code):
    return """SELECT gpnt_location FROM business WHERE postal_code = {} LIMIT 1""".format(postal_code)

In [116]:
select_gpnt_for_postal_code(92672)

'SELECT gpnt_location FROM business WHERE postal_code = 92672 LIMIT 1'

In [117]:
def match_gpnt(postal_code, distance):
    return """
            SELECT *
            FROM business
            WHERE ST_Distance_Sphere(gpnt_location, ({})) <= {}
            """.format(select_gpnt_for_postal_code(postal_code), distance)

In [120]:
print(match_gpnt(92672, 100))


            SELECT *
            FROM business
            WHERE ST_Distance_Sphere(gpnt_location, (SELECT gpnt_location FROM business WHERE postal_code = 92672 LIMIT 1)) <= 100
            


In [121]:
db.query_to_dataframe(match_gpnt(92672, 100))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1476 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,1048,37.770068,-122.446644,CREPE EXPRESS,+14158650264,94117,CA
1,1509 HAIGHT ST,San Francisco,0101000020E6100000B8AF03E78CE24240DFDDCA129D9C...,4257,37.769925,-122.447087,DELUXE,+14155526949,94117,CA
2,1480 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,4896,37.770068,-122.446644,BEN & JERRY'S HAIGHT-ASHBURY,+14154573140,94117,CA
3,1535 HAIGHT ST,San Francisco,0101000020E6100000AD3594DA8BE2424040852348A59C...,36635,37.769893,-122.447588,BIG SLICE PIZZA,+14155522520,94117,CA
4,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,92672,CA
5,1599 HAIGHT ST,San Francisco,0101000020E61000006DAE9AE788E24240B1FB8EE1B19C...,67587,37.769803,-122.448357,BURGER URGE,+14155220122,94117,CA
6,1568 HAIGHT ST,San Francisco,0101000020E61000009BCAA2B08BE242404224438EAD9C...,70970,37.769888,-122.448093,H.Q. BAR & RESTAURANT,,94117,CA


In [122]:
update_92672 = """
BEGIN;
UPDATE business SET postal_code = 94117 WHERE postal_code = 92672;
COMMIT;
"""

In [123]:
db.query_to_dictionary(update_92672, fetch_res=False)

In [124]:
db.query_to_dataframe(match_gpnt(94117, 100))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1530 HAIGHT ST,San Francisco,0101000020E6100000C32973F38DE242405FB87361A49C...,64660,37.769957,-122.447533,HAIGHT STREET MARKET,+14152550643,94117,CA
1,1476 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,1048,37.770068,-122.446644,CREPE EXPRESS,+14158650264,94117,CA
2,1509 HAIGHT ST,San Francisco,0101000020E6100000B8AF03E78CE24240DFDDCA129D9C...,4257,37.769925,-122.447087,DELUXE,+14155526949,94117,CA
3,1480 HAIGHT ST,San Francisco,0101000020E61000001BD9959691E24240193DB7D0959C...,4896,37.770068,-122.446644,BEN & JERRY'S HAIGHT-ASHBURY,+14154573140,94117,CA
4,1535 HAIGHT ST,San Francisco,0101000020E6100000AD3594DA8BE2424040852348A59C...,36635,37.769893,-122.447588,BIG SLICE PIZZA,+14155522520,94117,CA
5,1599 HAIGHT ST,San Francisco,0101000020E61000006DAE9AE788E24240B1FB8EE1B19C...,67587,37.769803,-122.448357,BURGER URGE,+14155220122,94117,CA
6,1568 HAIGHT ST,San Francisco,0101000020E61000009BCAA2B08BE242404224438EAD9C...,70970,37.769888,-122.448093,H.Q. BAR & RESTAURANT,,94117,CA


### Check the Status of our Bad Postal Codes

In [125]:
db.query_to_dataframe(select_postal_code_by_postal_code_less_than_10)

,postal_code
0,0
1,94013
2,94101
3,94120
4,94130
5,94140
6,94143
7,94513
8,94609
9,95105


In [126]:
db.query_to_dataframe(match_gpnt(94101, 100))

,address,city,gpnt_location,id,latitude,longitude,name,phone_number,postal_code,state
0,1555 MISSION ST,San Francisco,0101000020E610000082C476F700E34240F06B2409C29A...,1216,37.773467,-122.418093,ZAOH RESTAURANT,+14154313930,94103,CA
1,1455A MARKET ST,San Francisco,0101000020E6100000BEC1172653E34240962023A0C29A...,4661,37.775975,-122.418129,HAPPY DONUT,+14152528868,94103,CA
2,1455 MARKET ST 1/F,San Francisco,0101000020E61000005EB9DE3653E34240DEE68D93C29A...,5018,37.775977,-122.418126,MCDONALDS,+14154686990,94103,CA
3,1559 MISSION ST,San Francisco,0101000020E61000003B18B14F00E3424025ADF886C29A...,36745,37.773447,-122.418123,SHANGHAI CHINA RESTAURANT,+14156218188,94103,CA
4,41 MONTGOMERY ST,San Francisco,0101000020E6100000923B6C2233E34240C53C2B69C59A...,65856,37.774998,-122.418299,BAMBOO ASIA,+14156246790,94101,CA
5,1400 MARKET ST,San Francisco,0101000020E61000006D54A70359E3424008C89750C19A...,66219,37.776154,-122.418049,LITTLE GRIDDLE,+14158644292,94102,CA
6,1408 MARKET ST,San Francisco,0101000020E61000004A7EC4AF58E3424032755776C19A...,69121,37.776144,-122.418058,"MAVELOUS, LLC",+14156268884,94102,CA


## Plot locations

Next, we will use Folium to plot the businesses for which we have a latitude and a longitude. 

### `folium.Map` 

To build our `folium.Map` object we will need to center the map in a given location. We will use the `avg` aggregate function to do this. 

In [127]:
import folium

In [128]:
avg_location_sf = """
SELECT avg(latitude) as avg_lat, 
       avg(longitude) as avg_lon FROM ({}) q;
""".format(select_business_valid)

In [129]:
db.query_to_dataframe(avg_location_sf)

,avg_lat,avg_lon
0,37.774003,-122.425144


In [130]:
avg_loc_df = db.query_to_dataframe(avg_location_sf)

Then assign the result as a single list value.

In [131]:
avg_loc = avg_loc_df.values.tolist()[0]

In [132]:
avg_loc

[37.7740031665327, -122.425144254083]

In [133]:
sf_map = folium.Map(location=avg_loc, zoom_start=13)

In [134]:
within_200_meters_of_94101 = db.query_to_dataframe(match_gpnt(94101, 200))

In [135]:
near_94101_lat_long = within_200_meters_of_94101[['latitude', 'longitude','postal_code']].values.tolist()

In [136]:
for loc in near_94101_lat_long:
    try:
        postal_code = str(int(loc[2]))
    except:
        postal_code = None
    folium.Marker(loc[:2],postal_code).add_to(sf_map)

In [137]:
sf_map

## Maybe that one is OK

# Practice 

See if you can clean up the rest of the "bad" postal codes. If the postal codes are valid, keep them.

# What about those Postal Code `NaN`s?

Train a K Nearest Neighbors Model to Predict the NaN values. 

1. Pull all of the data with a valid lat, long, and postal code.
2. Perform a train test split on the data.
3. Fit a K Nearest Neighbors Model on the training data.
4. Validate the model using the test data. 
5. (optional) Performa Grid Search over the data to find the best number of neighbors to use.
6. (optional) Plot a complexity curve showing your model's accuracy versus the number of neighbors.
7. Pull all of the data with a valid lat, long, but no postal code. 
8. Use your model to assign a postal code to the data. 

# Data Analysis

Consider the `violation` table.


Answer these questions using sql:
1. How many violations are there?
- How many businesses committing violations?
- What's the average number of violations per business?

Answer these questions using python
1. Draw a plot of the violations count
- Is the average number of violations meaningful?
> Not really, the distribution is quite skewed
- draw a plot of the normalized cumulative violation counts. Can we discard the restaurants with few violations?
- where would you draw a threshold if you were to keep 90% of the violations?

Consider the `inspection` table.

Answer these questions using SQL:
1. What's the average score for the whole city?
- What's the average score per business?
- Does the score correlate with the number of inspections?
- Create a dataframe from a table with the following columns:
    business_id, average_score, number_of_inspections, number_of_violations
- Use pandas to do a scatter matrix plot of average_score, number_of_inspections, number_of_violations to check for correlations

### Postal Code Analysis

The town administration would like to know which zip code are the ones where they should focus the inspections.

Use the information contained in the `business` table as well as the previous tables to answer the following questions using SQL:

1. Count the number of businesses per zipcode and sort them by descending order
- Which are the top 5 zipcodes with the worst average score?
    - Restrict your analysis to the zipcodes with at least 50 businesses
    - Do a simple average of the inspections scores in the postal code
- Which are the top 5 zipcodes with the highest number of violations per restaurant?
    - Restrict your  analysis to the zipcodes with at least 50 businesses



## Final recommendation
Give a final recommendation on which 2 zipcodes should the administration focus and choose an appropriate plot to 